In [9]:
import hashlib
import requests

import pandas as pd

from bs4 import BeautifulSoup
from feedgen.feed import FeedGenerator
from flask import Flask, make_response
from datetime import date, datetime, timezone

In [7]:
URLO = "https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25"
    
RESP=requests.get(URLO)
HTML=RESP.text

SOUP = BeautifulSoup(HTML, "lxml")
ARTS = SOUP.find_all("div", {"class": "section__content--markdown section__content--markdown--neutral-article"})

In [20]:
dt = datetime.now()
dt = dt.replace(tzinfo=timezone.utc)
dt = dt.strftime("%a, %d %b %Y %H:%M:%S %z")

'Mon, 09 Jan 2023 11:54:19 +0000'

In [27]:
# generate data
COL = ['Date','Title','Link','Description','GUID']
DFO = pd.DataFrame(columns=COL)

for ART in ARTS:
    if len(ART) > 1:

        tt = ART.select('h2,h3')

        if tt:
            dt = dt
            
            tt = tt[0].text 

            try:
                lk = ART.find_all('a', href=True)[-2]
                lk = lk.get('href')
            except:
                lk = None

            ds = ART.text

            gd = 'guid{0}'.format(tt)
            gd = str(int(hashlib.sha256(guid.encode('utf-8')).hexdigest(), 16) % 10**8 + 1)
            
            row = [dt, tt, lk, ds, gd]
            DFR = pd.DataFrame([row], columns=COL)

            DFO = pd.concat([DFO,DFR], axis=0)
            
DFO = DFO.reset_index(drop=True).copy()
DFO.head()

,Date,Title,Link,Description,GUID
0,"Mon, 09 Jan 2023 11:54:19 +0000",Cost of future fixed-rate mortgages could incr...,/generator?uri=/economy/economicoutputandprodu...,\n\nCost of future fixed-rate mortgages could ...,72843048
1,"Mon, 09 Jan 2023 11:54:19 +0000",Post-Christmas retail activity rose compared w...,https://www.ons.gov.uk/releases/economicactivi...,\n\nPost-Christmas retail activity rose compar...,72843048
2,"Mon, 09 Jan 2023 11:54:19 +0000",The majority of adults planning to cut spendin...,https://www.ons.gov.uk/peoplepopulationandcomm...,\n\nThe majority of adults planning to cut spe...,72843048
3,"Mon, 09 Jan 2023 11:54:19 +0000",Rising food prices leave most vulnerable worst...,https://www.ons.gov.uk/releases/risingfoodpric...,\n\nRising food prices leave most vulnerable w...,72843048
4,"Mon, 09 Jan 2023 11:54:19 +0000",Business investment lower than previously esti...,https://www.ons.gov.uk/releases/businessinvest...,\n\nBusiness investment lower than previously ...,72843048


In [25]:
#initialise feed
fg = FeedGenerator()
fg.title('UK Economy RSS')
fg.description('RSS feed capturing updates to the ONS UK Economy blog')
fg.link(href='https://93b9-51-141-125-233.eu.ngrok.io')
#fg.image(url_for('static', filename='icon.png'))

[{'href': 'https://93b9-51-141-125-233.eu.ngrok.io'}]

In [32]:
for index, row in DFO.iterrows():
    #print(row)
    fe = fg.add_entry()
    
    fe.title(row.Title)
    fe.description(row.Date)
    fe.link(href=str(row.Link))
    
    fe.guid(row.GUID, permalink=False)
    fe.pubDate(row.Date)
    fe.author(name="Gregory Weegenaar", email="Gregory.Weegenaar@tdrcapital.com")
    

In [39]:
XML = fg.rss_str()
XML = XML.decode("utf-8") 

In [41]:
with open('readme.txt', 'w') as f:
    f.write(XML)

In [2]:
from xml.dom import minidom
import xml.etree.ElementTree as ET

In [6]:
PATH = r'C:\Users\DS.GWeegenaar\Desktop\Python\sandbox_actions\misc\rss\UK_economy_rss.xml'
with open(PATH, 'r', encoding='utf-8') as f:
    data = f.read()

In [14]:
def transform_xml(XML):
    TREE = ET.ElementTree(ET.fromstring(XML))
    ROOT = TREE.getroot()
    XMLO = minidom.parseString(ET.tostring(ROOT)).toprettyxml(indent="   ")
    return XMLO

In [20]:
with open(PATH, "w", encoding = "utf-8") as f:
    f.write(xmlstr)

In [ ]:
# moody downgrade
# moody negative outlook
# fitch downgrade


In [106]:
import re
import js_regex

import json
import requests

import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Moody's downgrades Spin Holdco's CFR to Caa1; outlook stable")

In [107]:
from spacy import displacy
#Set some display options for the visualizer
options = {"compact": True, "distance": 90, "color": "yellow", "bg": "black", "font": "Gill Sans"}

displacy.render(doc, style="dep", options=options)

In [100]:
base_endpoint = "https://serpapi.com/search.json?"
input_query = "allintitle:moody downgrades to site:https://www.moodys.com/"
input_params = {
                "q": input_query,
                "tbm": "nws",
                "num": "100",
                "location": "London",
                "api_key": "ae5695fed402b38494d25f4df01dc61422facdeb77e7b6864d368abdc16c96f5"
                }

response_json = requests.get(base_endpoint, params=input_params).json()

In [105]:
regex_title  = re.compile("Moody's downgrades.*")
regex_rating = js_regex.compile("to .*\d|to '(.*?)'")
regex_company = js_regex.compile("downgrades.*to")

for result in response_json['news_results'][0:5]:
    result_title = result['title']
    if regex_title.search(result_title):

        try:
            rating_raw = regex_rating.search(result_title)[0]
            rating_raw = rating_raw.replace("to ","")
        except:
            rating_raw = 'unknown'        
        try:
            rating_company = regex_company.search(result_title)[0]
            rating_company = re.sub("downgrades|to|ratings?|CFR|PDR|  ","", rating_company)
        except:
            rating_company = 'unknown'
        
        print(result_title) #print(rating_raw, '-->',rating_company, '-->', result_title)

Moody's downgrades City of Hope (CA) to A3; outlook stable
Moody's downgrades TortoiseEcofin's CFR to Caa1; outlook ...
Moody's downgrades Tacora Resources' rating to Caa3 ...
Moody's downgrades Takko's CFR to Caa3, outlook negative
Moody's downgrades Spin Holdco's CFR to Caa1; outlook stable


'to A3'